In [1]:
import os
import json
import random
import colorsys
import webcolors
import numpy as np
import pandas as pd
from sklearn.cluster import KMeans
from PIL import Image, ImageOps, ImageEnhance

In [18]:
# Requires a modified webcolors _conversion and _init__ code

css_color_map = webcolors.get_color_map(webcolors.CSS3)
css_rgbs = []
html_color_map = webcolors.get_color_map(webcolors.HTML4)
html_rgbs = []
for key in css_color_map.keys():
    css_rgbs.append(webcolors.hex_to_rgb(key))
for key in html_color_map.keys():
    html_rgbs.append(webcolors.hex_to_rgb(key))

css_numpy_arr = np.array([[v[0], v[1], v[2]] for v in css_rgbs])
html_numpy_arr = np.array([[v[0], v[1], v[2]] for v in html_rgbs])
color_arrays = {webcolors.CSS3: css_numpy_arr, webcolors.HTML4: html_numpy_arr}


def euclidean_distance(a, b):
    return np.linalg.norm(a - b, axis=-1)


def manhattan_distance(a, b):
    return np.sum(np.abs(a - b), axis=-1)


def closest_color(rgb_color, color_base=webcolors.CSS3):
    min_colors = {}
    for key, name in webcolors.get_color_map(color_base).items():
        r_c, g_c, b_c = webcolors.hex_to_rgb(key)
        rd = (r_c - rgb_color[0]) ** 2
        gd = (g_c - rgb_color[1]) ** 2
        bd = (b_c - rgb_color[2]) ** 2
        min_colors[(rd + gd + bd)] = name
    return min_colors[min(min_colors.keys())]


def get_color_name(rgb_tuple, base_color=webcolors.CSS3):
    try:
        # Convert RGB to hex
        hex_value = webcolors.rgb_to_hex(rgb_tuple)
        # Get the color name directly
        return webcolors.hex_to_name(hex_value, base_color)
    except ValueError:
        # If exact match not found, find the closest color
        return closest_color(rgb_tuple, base_color)
    

def find_closest_points_euclidean(image_array, color_base):
    color_dict = dict()
    for n_point in image_array:
        for pixel in n_point:
            # Compute Euclidean distance to all points in M
            distances = euclidean_distance(color_arrays[color_base], pixel)
            # Get index of the minimum distance
            closest_index = np.argmin(distances)
            closest_color = color_arrays[color_base][closest_index]
            color_name = webcolors.rgb_to_name((closest_color[0], closest_color[1], closest_color[2]), color_base)
            color_dict[color_name] = color_dict.get(color_name, 0) + 1
    return color_dict

In [3]:
''' 
color_ranges = {
    'red':    [[200, 0, 0], [255, 100, 100]],
    'green':  [(0, 200, 0), (100, 255, 100)],
    'blue':   [(0, 0, 200), (100, 100, 255)],
    'yellow': [(200, 200, 0), (255, 255, 100)],
    'cyan':   [(0, 200, 200), (100, 255, 255)],
    'magenta': [(200, 0, 200), (255, 100, 255)],
    'orange': [(200, 100, 0), (255, 200, 80)],  # Orange range
    'indigo': [(75, 0, 130), (100, 50, 150)],   # Indigo range
    'black':  [(0, 0, 0), (50, 50, 50)],
    'white':  [(200, 200, 200), (255, 255, 255)]
    } 
'''

def resize_image(img, width=180, height=240):
    resized_image = img.resize((width, height)).convert('RGB')
    return resized_image

def add_border(img, color=(0, 0, 0), border_width=10):
    return ImageOps.expand(img, border=border_width, fill=color)

def get_avg_rgb(img):
    image_np = np.array(img)
    avg_color = np.mean(image_np, axis=(0, 1), dtype=int)  # Average across pixels
    return avg_color

def rgb_to_hsv(rgb):
    r, g, b = rgb / 255.0
    h, s, v = colorsys.rgb_to_hsv(r, g, b)
    if s < 0.01:  # Close to 0 saturation means black, white, or gray
        h = -1 
    else:
        h *= 360
    return h, s, v

def rgb_to_hls(rgb):
    r, g, b = rgb / 255.0
    h, l, s = colorsys.rgb_to_hls(r, g, b)
    if s < 0.01:  # Close to 0 saturation means black, white, or gray
        h = -1 
    else:
        h *= 360
    return h, l, s

In [16]:
def count_image_colors_numpy(img):
    css_color_dict = find_closest_points_euclidean(img, webcolors.CSS3)
    html_color_dict = find_closest_points_euclidean(img, webcolors.HTML4)
    return css_color_dict, html_color_dict


def count_image_colors(img):
    css_color_dict = dict()
    html_color_dict = dict()
    for row in img:
        for pixel in row:
            css_color_name = get_color_name(pixel, webcolors.CSS3)
            html_color_name = get_color_name(pixel, webcolors.HTML4)
            css_color_dict[css_color_name] = css_color_dict.get(css_color_name, 0) + 1
            html_color_dict[html_color_name] = html_color_dict.get(html_color_name, 0) + 1
    return css_color_dict, html_color_dict

In [21]:
image_list = os.listdir('Covers 180/')[:1]
tt = count_image_colors_numpy(np.asarray(Image.open('Covers 180/' + image_list[0]), dtype=int))

In [10]:
image_list = os.listdir('Covers/')
# images_objs = [Image.open('Covers 1200/' + img) for img in image_list]

In [ ]:
with open('ultimate_games.json', 'r', encoding='utf-8') as file:
    ultimate_games = json.load(file)
with open('game_platform_franchise.txt', 'r', encoding='utf-8') as file:
    franchise_games = file.readlines()

clean_images_list = [v.replace('_Cover.jpg', '') for v in image_list]

In [13]:
image_dict = dict()
for line in franchise_games:
    line_feat = line.replace('\n', '').split('\t')
    line_franchise_1 = line_feat[0]
    line_franchise_2 = line_feat[1]
    line_developer = line_feat[2]
    game_title = line_feat[3]
    image_title = line_feat[4]
    
    if line_franchise_2 != '':
        if line_franchise_2 not in image_dict:
            image_dict[line_franchise_2] = []
        image_dict[line_franchise_2].append(image_title)
        continue

    if line_franchise_1 != '':
        if line_franchise_1 not in image_dict:
            image_dict[line_franchise_1] = []
        image_dict[line_franchise_1].append(image_title)
        continue

    if line_developer != '':
        if line_developer not in image_dict:
            image_dict[line_developer] = []
        image_dict[line_developer].append(image_title)
        continue

    image_dict[game_title] = [image_title]

In [14]:
sorted_images = []
for v in sorted(list(image_dict.values()), key=len, reverse=True):
    sorted_images.extend(v)

for ind in range(len(sorted_images)):
    sorted_images[ind] += '_Cover.jpg'

images_objs = [Image.open('Covers/' + img) for img in sorted_images]


In [14]:
for img in image_list:
    img_obj = Image.open('Covers/' + img)
    for sz in [(864, 1269), (1200, 1600), (180, 240)]:
        resized_image = resize_image(img_obj, sz[0], sz[1])
        resized_image.save('Covers ' + str(sz[0]) + '/' + img.replace('_Cover', ''))

In [ ]:
horizontal_count = 4
image_hor_list = []
for offset in range(0, len(images_objs), horizontal_count):
    image_hor_list.append(np.hstack([img for img in images_objs[offset: offset+horizontal_count]]))

In [18]:
vertical_count = 4
image_ver_list = []
for offset in range(0, len(image_hor_list), vertical_count):
    image_ver_list.append(np.vstack([img for img in image_hor_list[offset: offset+vertical_count]]))
    # new_image.save('Posters/Trifecta_vertical.jpg')

In [19]:
for ind, img in enumerate(image_ver_list):
    imgs_poster = Image.fromarray(img)
    imgs_poster.save('Posters/poster_' + str(ind) + '.jpg')

In [15]:
image_list = os.listdir('Covers Best/')
poster_size = 16
random.shuffle(image_list)
poster_count = 5000
for offset in range(0, len(image_list), poster_size):
    images_objs = [Image.open('Covers Best/' + img) for img in image_list[offset: offset+poster_size]]
    converters = [ImageEnhance.Color(v) for v in images_objs]
    image_obj = [v.enhance(1.5) for v in converters]
    images_objs = [ImageOps.expand(img, border=20, fill='yellow') for img in images_objs]
    horizontal_count = 4
    image_hor_list = []
    for offset in range(0, len(images_objs), horizontal_count):
        image_hor_list.append(np.hstack([img for img in images_objs[offset: offset+horizontal_count]]))

    poster = np.vstack([img for img in image_hor_list])
    poster = Image.fromarray(poster)
    poster.save('Posters/poster_' + str(poster_count) + '.jpg')
    poster_count += 1

In [ ]:
image_list = os.listdir('Covers 180/')
color_info_dict = dict()
for ind, img in enumerate(image_list):
    print(img)
    image_obj = Image.open('Covers 180/' + img)
    image_obj = np.asarray(image_obj, dtype=int)
    r_g_b = get_avg_rgb(image_obj)
    h1, s1, v1 = rgb_to_hsv(r_g_b)
    _, l1, s2 = rgb_to_hls(r_g_b)
    css_dict, html_dict = count_image_colors_numpy(image_obj)
    color_info_dict[img] = {'r': r_g_b[0], 'g': r_g_b[1], 'b': r_g_b[2],
                            'hue': round(h1, 2), 'sat_1': round(s1, 2), 'value': round(v1),
                            'lumo': round(l1, 2), 'sat_2': round(s2, 2)}
    color_info_dict[img]['css'] = css_dict
    color_info_dict[img]['html'] = html_dict

'''
for ind, arr in enumerate(color_mean_list):
    hsv = colorsys.rgb_to_hsv(arr[0], arr[1], arr[2])
    hsl = colorsys.rgb_to_hls(arr[0], arr[1], arr[2])
    print(ind, arr[0], arr[1], arr[2], hsv[0], hsv[1], hsv[2], hsl[0], hsl[1], hsl[2])
'''

In [ ]:
new_dict = dict()
for game, val in color_info_dict.items():
    new_dict[game] = json.loads(str(val).replace("'", '"'))
with open('cover_colors.json', 'w', encoding='utf-8') as file:
    json.dump(new_dict, file)

{'r': 130, 'g': 130, 'b': 130, 'hue': -1, 'sat_1': 0.0, 'value': 1, 'lumo': 0.51, 'sat_2': 0.0, 'css': {'white': 8025, 'whitesmoke': 2474, 'snow': 1412, 'silver': 1584, 'gray': 2131, 'darkgray': 4970, 'lightgray': 1145, 'ghostwhite': 324, 'gainsboro': 1544, 'lavender': 452, 'lightslategray': 1280, 'dimgray': 1734, 'black': 15123, 'darkslategray': 1002}, 'html': {'white': 13263, 'silver': 4833, 'gray': 8772, 'pink': 582, 'black': 15750}}


In [2]:
rgb_avg = pd.read_csv('Colors.csv', index_col=False).values

In [5]:
image_list = os.listdir('Covers 864/')
colors_ds = pd.read_csv('colors-manual-order.csv', index_col=None).values

for ind, image in enumerate(colors_ds):
    image_path = image[0] + '.jpg'
    image_obj = Image.open('Covers 864/' + image_path)
    image_obj = ImageOps.expand(image_obj.resize((864, 1269)).convert('RGB'), border=20, fill='red')
    # converter = ImageEnhance.Color(image_obj)
    # image_obj = converter.enhance(1.5)
    image_obj.save('Covers Test/' +  str(ind+1) + ' ' + image[0] + '.jpg')

In [ ]:
image_list = os.listdir('Covers 180/')
for image in image_list:
    image_obj = Image.open('Covers 180/' + image)
    image_obj = np.asarray(image_obj)

In [68]:
def get_image_pixel_features(image_path):
    """
    Extract all pixel RGB values from an image as the feature set.
    This will flatten the image into a 2D array of RGB pixels.
    """
    image = Image.open(image_path).convert('RGB')
    image_np = np.array(image)  # Convert to NumPy array
    return image_np.reshape(-1, 3)  # Flatten to a 2D array of pixels (N, 3)


def cluster_images_by_pixels(folder_path, n_clusters=5):
    """
    Cluster images based on the RGB values of all pixels using k-means.
    :param folder_path: Path to the folder containing images.
    :param n_clusters: Number of clusters for pixel-level k-means.
    """
    image_files = [f for f in os.listdir(folder_path) if f.endswith(('jpg', 'png', 'jpeg'))]
    image_features = []  # Store features for each image

    for img in image_files:
        img_path = os.path.join(folder_path, img)
        pixel_features = get_image_pixel_features(img_path)  # Get all pixel colors
        kmeans_pixels = KMeans(n_clusters=5, random_state=42).fit(pixel_features)
        cluster_centers = kmeans_pixels.cluster_centers_  # Centroids represent dominant colors
        image_features.append(cluster_centers.flatten())  # Flatten for clustering images
    
    # Cluster images based on their pixel-level dominant colors
    image_features = np.array(image_features)
    kmeans_images = KMeans(n_clusters=n_clusters, random_state=42).fit(image_features)
    labels = kmeans_images.labels_
    
    # Organize images into clusters
    for cluster_idx in range(n_clusters):
        cluster_folder = os.path.join(folder_path, f"image_cluster_{cluster_idx}")
        os.makedirs(cluster_folder, exist_ok=True)
        
        for img, label in zip(image_files, labels):
            if label == cluster_idx:
                img_path = os.path.join(folder_path, img)
                output_path = os.path.join(cluster_folder, img)
                Image.open(img_path).save(output_path)

# Usage
cluster_images_by_pixels('Covers 180/', n_clusters=20)